In [1]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from regime_definitions import compute_market_volatility_regime, compute_cross_sectional_dispersion_regime
from data_loader import load_price_data
import pandas as pd
import numpy as np

C:\Users\edzz0\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
price = pd.read_parquet("../data/processed/price_filtered.parquet")

regime_vol = compute_market_volatility_regime(price)
regime_disp = compute_cross_sectional_dispersion_regime(price)

regime_vol.to_frame(name="vol_regime").to_parquet("../data/processed/regime_market_vol.parquet")
regime_disp.to_frame(name="disp_regime").to_parquet("../data/processed/regime_cross_dispersion.parquet")

In [3]:
price.shape

(3862, 419)

In [19]:
regime_vol

date
2010-01-04     neutral
2010-01-05     neutral
2010-01-06     neutral
2010-01-07     neutral
2010-01-08     neutral
                ...   
2025-05-05    high_vol
2025-05-06    high_vol
2025-05-07    high_vol
2025-05-08    high_vol
2025-05-09    high_vol
Length: 3862, dtype: object